    Atividade
        ● Selecione uma série temporal
        ● Implemente 5 preditores (variando parâmetros ou modelos)
        ● A combinação é melhor do que os preditores individuais? Em que proporção?
            ○ Utilize inicialmente a média artimética simples, mas depois experimente o uso da média winsorizada

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.svm import SVR
from sklearn import metrics
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
def winsor_mean(lista):
    lista.sort()
    lista_nova = [x for x in lista if x != lista[0] and x != lista[-1]]
    return np.mean(lista_nova)

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [4]:
def metricas(y_test, predict):
    return metrics.mean_absolute_error(y_test, predict)

In [5]:
pip_LogisticRegression = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', LogisticRegression())
])

In [6]:
pip_SVR = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVR(kernel= 'linear',C=0.1,gamma=0.0001))
])

In [7]:
pip_SVR_parametro = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVR(kernel= 'rbf',C=1,gamma=0.001)) 
])

In [8]:
pip_SVR_parametro1 = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVR(kernel= 'poly',C=10,gamma=0.01))
])

In [9]:
pip_SVR_parametro2 = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVR(kernel= 'sigmoid',C=100,gamma=0.001))
])

In [10]:
dados = pd.read_csv('CSVs/AirPassengers.csv', sep=',', header=0, index_col='Month')

In [11]:
janela = series_to_supervised(data=dados, n_in=3, n_out=12)
y = janela['var1(t)']
x = janela[['var1(t-3)', 'var1(t-2)', 'var1(t-1)']]

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

                                      LogisticRegression
   ========================================================================================================

In [13]:
pip_LogisticRegression.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [14]:
predicao_logistic = pip_LogisticRegression.predict(x_test)

In [15]:
print("MAE:",metricas(y_test, predicao_logistic))

MAE: 51.15384615384615


                                             SRV
  =============================================================================================================

In [16]:
pip_SVR.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0001,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False))])

In [17]:
predicao_svr = pip_SVR.predict(x_test)

In [18]:
print("MAE:",metricas(y_test,predicao_svr))

MAE: 57.91862642313186


In [19]:
pip_SVR_parametro.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))])

In [20]:
predicao_parametro = pip_SVR_parametro.predict(x_test)

In [21]:
print("MAE:",metricas(y_test,predicao_parametro))

MAE: 72.90072348050673


In [22]:
pip_SVR_parametro1.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.01,
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False))])

In [23]:
predicao_parametro1 = pip_SVR_parametro1.predict(x_test)

In [24]:
print("MAE:",metricas(y_test, predicao_parametro1))

MAE: 73.17173824953727


In [25]:
pip_SVR_parametro2.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
  kernel='sigmoid', max_iter=-1, shrinking=True, tol=0.001, verbose=False))])

In [26]:
predicao_parametro2 = pip_SVR_parametro2.predict(x_test)

In [27]:
print("MAE:",metricas(y_test, predicao_parametro2))

MAE: 57.918737410337904


In [28]:
media_simples = []
for media in range(len(y_test)):
    media_simples.append(np.mean([predicao_logistic[media],predicao_svr[media], 
                                  predicao_parametro[media],predicao_parametro1[media],predicao_parametro2[media]])
                        )
    
mae_media_simples = metrics.mean_absolute_error(media_simples, y_test)
print('MAE Média Simples = ', mae_media_simples)

MAE Média Simples =  53.244811509488514


In [52]:
winsore = winsorize([predicao_logistic,
                     predicao_svr,
                     predicao_parametro,
                     predicao_parametro1,
                     predicao_parametro2])
media_winsorizada = np.mean(winsore)
print("Média Winsorizada",media_winsorizada)

Média Winsorizada 244.84370092969195


In [57]:
dados_comparacao = {
    'MODELO':['Logistic Regression', 'SVR', 'SVR1', 'SVR2', 'SVR3','MEDIA SIMPLES','MEDIA WINSORIZADA'],
    'MAE':[metricas(y_test, predicao_logistic),
           metricas(y_test,predicao_svr),
           metricas(y_test,predicao_parametro),
           metricas(y_test, predicao_parametro1),
           metricas(y_test, predicao_parametro2),
           mae_media_simples,
           media_winsorizada],
    'Prop Média Simples':[metricas(y_test, predicao_logistic)/mae_media_simples,
                     metricas(y_test,predicao_svr)/mae_media_simples,
                     metricas(y_test,predicao_parametro)/mae_media_simples,
                     metricas(y_test, predicao_parametro1)/mae_media_simples,
                     metricas(y_test, predicao_parametro2)/mae_media_simples,
                     media_winsorizada/mae_media_simples,media_winsorizada/mae_media_simples],
    'Média Winsorizada':[metricas(y_test, predicao_logistic)/media_winsorizada,
                         metricas(y_test,predicao_svr)/media_winsorizada,
                         metricas(y_test,predicao_parametro)/media_winsorizada,
                         metricas(y_test, predicao_parametro1)/media_winsorizada,
                         metricas(y_test, predicao_parametro2)/media_winsorizada,
                         mae_media_simples/media_winsorizada,mae_media_simples/media_winsorizada]
}
df_comparacao = pd.DataFrame(data=dados_comparacao)

In [58]:
df_comparacao

,MODELO,MAE,Prop Média Simples,Média Winsorizada
0,Logistic Regression,51.153846,0.960729,0.208924
1,SVR,57.918626,1.087780,0.236553
2,SVR1,72.900723,1.369161,0.297744
3,SVR2,73.171738,1.374251,0.298851
4,SVR3,57.918737,1.087782,0.236554
5,MEDIA SIMPLES,53.244812,4.598452,0.217464
6,MEDIA WINSORIZADA,244.843701,4.598452,0.217464
